In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()
import numpy as np
%matplotlib inline


In [2]:

listings = pd.read_csv('data/listings.csv')
calendar = pd.read_csv('data/calendar.csv')
reviews = pd.read_csv('data/reviews.csv')

data = [listings, calendar, reviews]

In [4]:

def obs_data(dfs): # function for quick observation of our data
    for df in dfs:
#         display(df.head())
        print('\n-----------------------Loading Next DF------------------------------\n')
        print('The shape of the dataframe is\n', df.shape, '\n')
        null_df = df[df.isna()]  # creating a df of columns records with null values
        null_cols = df.columns[df.isna().any()]  # creating a df of the column names to that are missing data 
        print('Names of Missing Columns')
        print(null_cols.tolist(), '\n')

        num_cols_mising = null_cols.nunique()
        print('Number of Missing Columns')
        print(num_cols_mising)

        # percent of missing columns
        tot_cols = df.columns.nunique()
        print('\nPercent of columns missing records (%)')
        print(np.round((1-((tot_cols-num_cols_mising)/tot_cols))*100))

        # Missing Rows ---------------------------
        num_missing_values = df.isna().sum()
#         print(num_missing_values.values)
        percent_missing_values = np.round(((1-(df.shape[0]-num_missing_values)/df.shape[0])*100),2)
#         # print(df.shape[0])
#         print(percent_missing_values)
        print('\nColumn','|' 'Percent of Missing Values (%)')
        print('----------------------------------- \n')
        for col, value in percent_missing_values.items():
            if value != 0:
        #         temp_dict = {'Columns': col, 'Percent Missing Records':values}
                print((col),'|',(value))
            else:
                pass
        print('n', df.describe())
    print('\n...')
    print('...')
    print('...\n')
    print('DONE!')



obs_data(data)
            


-----------------------Loading Next DF------------------------------

The shape of the dataframe is
 (46527, 74) 

Names of Missing Columns
['name', 'description', 'neighborhood_overview', 'host_name', 'host_since', 'host_location', 'host_about', 'host_response_time', 'host_response_rate', 'host_acceptance_rate', 'host_is_superhost', 'host_thumbnail_url', 'host_picture_url', 'host_neighbourhood', 'host_listings_count', 'host_total_listings_count', 'host_has_profile_pic', 'host_identity_verified', 'neighbourhood', 'bathrooms', 'bathrooms_text', 'bedrooms', 'beds', 'minimum_minimum_nights', 'maximum_minimum_nights', 'minimum_maximum_nights', 'maximum_maximum_nights', 'minimum_nights_avg_ntm', 'maximum_nights_avg_ntm', 'calendar_updated', 'first_review', 'last_review', 'review_scores_rating', 'review_scores_accuracy', 'review_scores_cleanliness', 'review_scores_checkin', 'review_scores_communication', 'review_scores_location', 'review_scores_value', 'license', 'reviews_per_month'] 

Numb

We can see some interesting insights from the intial analysis of this data. For example ...

In [7]:
# Now we can write an equation to clean the data for us

def clean_data(dfs):
    for df in dfs:
        df.dropna(thresh=(.3*df.shape[0]), axis=1, inplace=True) # dropping entire columns where the % of null values is greater than 30
        df.dropna(axis=0, inplace=True)  # drops the rows of all the other columns
        
        empty_check = len((df.columns[df.isna().any()]).tolist())  # checking for empty columns converting to list and then counting the length
        if empty_check == 0:
            print (df.isna().any())
            print('No null values')
        else:
            print(df.isna().sum())
            print('We missed something')
        
        
clean_data(data)

id                                              False
listing_url                                     False
scrape_id                                       False
last_scraped                                    False
name                                            False
                                                ...  
calculated_host_listings_count                  False
calculated_host_listings_count_entire_homes     False
calculated_host_listings_count_private_rooms    False
calculated_host_listings_count_shared_rooms     False
reviews_per_month                               False
Length: 71, dtype: bool
No null values
listing_id        False
date              False
available         False
price             False
adjusted_price    False
minimum_nights    False
maximum_nights    False
dtype: bool
No null values
listing_id       False
id               False
date             False
reviewer_id      False
reviewer_name    False
comments         False
dtype: bool
No null values


We have no missing values!